## **Solar+Flare v1.0**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.multioutput import MultiOutputClassifier

def cargar_datos(ruta):
    df = pd.read_csv(ruta, sep=r'\s+', header=None, skiprows=1).to_numpy()
    datos = np.array([fila[:10] for fila in df])
    clase = np.array([fila[-3:] for fila in df])
    return datos, clase

def codificar_datos(datos):
    label_encoders = []
    for i in range(datos.shape[1]):
        if isinstance(datos[0, i], str):
            le = LabelEncoder()
            datos[:, i] = le.fit_transform(datos[:, i])
            label_encoders.append(le)
        else:
            label_encoders.append(None)
    datos = datos.astype(int)
    return datos, label_encoders

ruta_local = "solar+flare/flare.data2"

try:
    datos, clase = cargar_datos(ruta_local)
    datos, label_encoders = codificar_datos(datos)
    for j in range(clase.shape[1]):
        if isinstance(clase[0, j], str):
            le = LabelEncoder()
            clase[:, j] = le.fit_transform(clase[:, j])
    clase = clase.astype(int)
except Exception as e:
    print(f"Ocurrió un error al leer el archivo: {e}")

datos_entrena, datos_prueba, clase_entrena, clase_prueba = train_test_split(datos, clase, test_size=0.2)

scaler = StandardScaler() 
datos_entrena = scaler.fit_transform(datos_entrena) 
datos_prueba = scaler.transform(datos_prueba)

modelo = MultiOutputClassifier(LogisticRegression(max_iter=200))
modelo.fit(datos_entrena, clase_entrena)
np.set_printoptions(suppress=True)

predicción = modelo.predict(datos_prueba)

for col, le in enumerate(label_encoders):
    if le is not None:
        print(f"Columna {col+1} clases: {le.classes_}")

print("Porcentaje de aciertos:", modelo.score(datos_prueba, clase_prueba))
print("\n", predicción)

## **Solar+Flare v2.0**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

nombres_columnas = ["modified Zurich class", "largest spot size", "spot distribution", 
                    "activity", "evolution", "previous 24 hour flare activity", 
                    "historically-complex", "became complex on this pass", 
                    "area", "area of largest spot", "common flares", 
                    "moderate flares", "severe flares"]

df = pd.read_csv("solar+flare/flare.data2", sep=r'\s+', header=None, skiprows=1, names=nombres_columnas)

#df.info()

df["modified Zurich class"] = df["modified Zurich class"].astype("category")
df["largest spot size"] = df["largest spot size"].astype("category")
df["spot distribution"] = df["spot distribution"].astype("category")

df_categorico = df[["modified Zurich class", "largest spot size", "spot distribution"]]

codificador = OneHotEncoder()
codificacion = codificador.fit_transform(df_categorico)

nuevas_columnas = pd.DataFrame(codificacion.toarray(), columns=codificador.get_feature_names_out(df_categorico.columns))

df = df.drop(["modified Zurich class", "largest spot size", "spot distribution"], axis="columns")
df = pd.concat([df, nuevas_columnas], axis="columns")

clase = df[["common flares", "moderate flares", "severe flares"]].to_numpy()
datos = df.drop(columns=["common flares", "moderate flares", "severe flares"]).to_numpy()

datos_entrena, datos_prueba, clase_entrena, clase_prueba = train_test_split(datos, clase, test_size=0.30)

scaler = StandardScaler()
datos_entrena = scaler.fit_transform(datos_entrena) 
datos_prueba = scaler.transform(datos_prueba)

modelo = MultiOutputClassifier(LogisticRegression(max_iter=1000, solver='saga'))
modelo.fit(datos_entrena, clase_entrena)
np.set_printoptions(suppress=True)

predicciones = modelo.predict(datos_prueba)
print(f'Precisión del modelo: {modelo.score(datos_prueba, clase_prueba)}\n')

predicciones_df = pd.DataFrame(predicciones, columns=["common flares (Pred)", "moderate flares (Pred)", "severe flares (Pred)"])
clase_prueba_df = pd.DataFrame(clase_prueba, columns=["common flares (Real)", "moderate flares (Real)", "severe flares (Real)"])
comparacion = pd.concat([clase_prueba_df.reset_index(drop=True), predicciones_df.reset_index(drop=True)], axis=1)
comparacion

## **Métricas**

In [ ]:
from sklearn.metrics import (confusion_matrix, accuracy_score, precision_score, recall_score, f1_score)
import matplotlib.pyplot as plt
import seaborn as sns

for i, output in enumerate(["common flares", "moderate flares", "severe flares"]):
    print(f"Métricas para {output}:")
    
    accuracy = accuracy_score(clase_prueba[:, i], predicciones[:, i])
    precision = precision_score(clase_prueba[:, i], predicciones[:, i], average='macro', zero_division=0)
    recall = recall_score(clase_prueba[:, i], predicciones[:, i], average='macro', zero_division=0)
    f1 = f1_score(clase_prueba[:, i], predicciones[:, i], average='macro', zero_division=0)
    
    print(f"Exactitud: {accuracy}")
    print(f"Precisión: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")

    conf_matrix = confusion_matrix(clase_prueba[:, i], predicciones[:, i])
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
    plt.title(f"Matriz de Confusión para {output}")
    plt.xlabel("Predicciones")
    plt.ylabel("Reales")
    plt.show()